In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# --- 1. Data Preparation ---
# Load and combine data
train = pd.read_csv('train_v9rqX0R.csv')
test = pd.read_csv('test_AbJTz2l.csv')
data = pd.concat([train.assign(source='train'), test.assign(source='test')], ignore_index=True)

# Impute Item_Weight
item_avg_weight = data.pivot_table(values='Item_Weight', index='Item_Identifier')
overall_avg_weight = data['Item_Weight'].mean()
def get_item_weight(item_id):
    if item_id in item_avg_weight.index:
        return item_avg_weight.loc[item_id, 'Item_Weight']
    return overall_avg_weight
data.loc[data['Item_Weight'].isnull(),'Item_Weight'] = data.loc[data['Item_Weight'].isnull(),'Item_Identifier'].apply(get_item_weight)

# Impute Outlet_Size
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x: x.mode()[0]) )
data.loc[data['Outlet_Size'].isnull(),'Outlet_Size'] = data.loc[data['Outlet_Size'].isnull(),'Outlet_Type'].apply(lambda x: outlet_size_mode[x])

# Impute Item_Visibility 0s
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')
overall_avg_visibility = data['Item_Visibility'].mean()
def get_avg_visibility(item_id):
    if item_id in visibility_avg.index:
        return visibility_avg.loc[item_id, 'Item_Visibility']
    return overall_avg_visibility
miss_bool = (data['Item_Visibility'] == 0)
data.loc[miss_bool,'Item_Visibility'] = data.loc[miss_bool,'Item_Identifier'].apply(get_avg_visibility)

# Feature Engineering
data['Item_Visibility_MeanRatio'] = data.apply(lambda x: x['Item_Visibility']/get_avg_visibility(x['Item_Identifier']), axis=1)
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
data['Item_Type_Combined'] = data['Item_Type'].apply(lambda x: 'Food' if x in ['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables', 'Baking Goods', 'Snack Foods', 'Frozen Foods', 'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned', 'Breads', 'Starchy Foods'] else ('Non-Consumable' if x in ['Household', 'Others'] else 'Drinks'))
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"

# Label Encoding
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mod:
    data[i] = le.fit_transform(data[i])

# One Hot Encoding
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])
for col in data.columns:
    if data[col].dtype == 'bool':
        data[col] = data[col].astype('float64')

# Final split and clean
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)
train = data.loc[data['source']=="train"].copy()
test = data.loc[data['source']=="test"].copy()
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

# Define features
predictors = [x for x in train.columns if x not in ['Item_Outlet_Sales', 'Item_Identifier','Outlet_Identifier']]
y = train['Item_Outlet_Sales']
X = train[predictors]
X_test = test[predictors]


# --- 2. FINAL MODEL DEFINITIONS ---

# 1. LGBM V3: Increased Complexity (from 64 to 128 leaves)
best_lgb_v3 = LGBMRegressor(
    subsample=0.8, reg_lambda=1, reg_alpha=0, num_leaves=128, n_estimators=3000,
    min_child_samples=50, max_depth=12, learning_rate=0.005, colsample_bytree=0.8,
    random_state=RANDOM_SEED, n_jobs=-1
)

# 2. XGBoost V3: Increased Complexity (from depth 8 to depth 9)
best_xgb_reg_v3 = XGBRegressor(
    subsample=0.8, reg_lambda=1.0, reg_alpha=0.1, n_estimators=2000,
    max_depth=9, learning_rate=0.01, colsample_bytree=0.6,
    random_state=RANDOM_SEED, n_jobs=-1, verbosity=0, tree_method="auto"
)

# 3. CatBoost V2 (Kept stable, as it performed well)
best_cat_v2 = CatBoostRegressor(
    iterations=3500, learning_rate=0.02, depth=6, eval_metric='RMSE',
    random_seed=RANDOM_SEED, early_stopping_rounds=100,
    logging_level='Silent', allow_writing_files=False
)

# 4. Random Forest (Reverting to safer settings to reduce noise)
rf = RandomForestRegressor(
    n_estimators=400, max_depth=6, min_samples_leaf=100,
    n_jobs=-1, random_state=RANDOM_SEED
)

estimators_v3 = [
    ('lgb', best_lgb_v3),
    ('xgb', best_xgb_reg_v3),
    ('cat', best_cat_v2),
    ('rf', rf)
]

stack_reg_v3 = StackingRegressor(
    estimators=estimators_v3,
    final_estimator=RidgeCV(alphas=[0.1, 1.0, 10.0]),
    passthrough=True,
    n_jobs=-1
)

print("Training Final Stacking Regressor")
stack_reg_v3.fit(X, y)


# --- 3. Prediction and Final Submission ---

# Predict test set
test_preds_stack_v3 = stack_reg_v3.predict(X_test)

# Clip to 0 for submission safety
final_preds_v3 = np.maximum(0, test_preds_stack_v3)

# Save submission
submission = test[['Item_Identifier','Outlet_Identifier']].copy()
submission['Item_Outlet_Sales'] = final_preds_v3
submission.to_csv("submission_final_stack.csv", index=False)
print("Saved submission_final_stack.csv")